In [10]:
import numpy as np
import pandas as pd

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from pylab import rcParams
import warnings
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore") 
%matplotlib inline

In [2]:
auto = pd.read_csv('Auto.csv', na_values='?')
auto.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      5
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [3]:
auto = auto.dropna()
auto.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [4]:
auto.shape[0]

392

In [5]:
auto = auto.drop(['name'], 1)
auto['horsepower'] = auto['horsepower'].convert_objects(convert_numeric = True)

In [6]:
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= auto['mpg'].median() else 0).astype('category')

In [7]:
Xvars = auto[['cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
yvals = auto['mpg_high'].values

In [8]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)
General_error_0 = np.zeros(k)
General_error_1 = np.zeros(k)
General_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    print('When k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE = pd.DataFrame({'error': (y_test != y_pred).astype(int), 
                        'y_pred': y_pred})
    error_0 = MSE[MSE['y_pred'] == 0]['error'].mean()
    error_1 = MSE[MSE['y_pred'] == 1]['error'].mean()
    error_all_class = MSE['error'].mean()
    General_error_0[k_ind] = error_0
    General_error_1[k_ind] = error_1
    General_MSE[k_ind] = error_all_class
    print('\n',classification_report(y_test, y_pred, digits=3))
    print('error rate (class 0) is', error_0,
          ', error rate (class 1) is', error_1)
    print('MSE (for all classes) is', error_all_class)
    print('The stds are respectively', MSE[MSE['y_pred'] == 0]['error'].std(),
          ',', MSE[MSE['y_pred'] == 1]['error'].std(),
          ',', MSE['error'].std(),'\n')    
    k_ind += 1


print('\nk-fold resuts:')
print('The average error rate (class 0) is', General_error_0.mean(), ', std is', General_error_0.std())
print('The average error rate (class 1) is', General_error_1.mean(), ', std is', General_error_1.std())
print('The average MSE (for all classes) is', General_MSE.mean(), ', std is', General_MSE.std())

When k index= 0

              precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

error rate (class 0) is 0.057692307692307696 , error rate (class 1) is 0.13043478260869565
MSE (for all classes) is 0.09183673469387756
The stds are respectively 0.2354354778987093 , 0.3405026123034997 , 0.29028015951453634 

When k index= 1

              precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

error rate (class 0) is 0.12244897959183673 , error rate (class 1) is 0.08163265306122448
MSE (for all classes) is 0.10204081632653061
The stds are respectively 0.331200660553418 , 0.27664166758624387 , 0.30425829769936363 

When k index= 2

              precision    recall  f1-score   support

          0

In [88]:
RF = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                           oob_score=True, random_state=25)

RF.fit(Xvars, yvals)
RF_pred = RF.oob_decision_function_

Since RandomForestClassifier does not have the attribute ".oob prediction_", I use oob_decision_function_ to calculate the OOB MSE. It is worth noting that when we use the oob_decision_function, we must transform some decimal values as 1 or 0.

In [92]:
MSE_RF = pd.DataFrame({'pred' : RF_pred.T[1], 'yvals': yvals})

In [93]:
MSE_RF['pred'] = MSE_RF['pred'].apply(lambda x: 1 if x >= 0.5 else 0)

In [95]:
MSE_all = mean_squared_error(MSE_RF['yvals'], MSE_RF['pred'])
MSE_all

0.07142857142857142

In [96]:
MSE_RF_0 = MSE_RF[MSE_RF['pred'] < 0.5]
MSE_0 = mean_squared_error(MSE_RF_0['yvals'], MSE_RF_0['pred'])
MSE_0

0.05789473684210526

In [97]:
MSE_RF_1 = MSE_RF[MSE_RF['pred'] >= 0.5]
MSE_1 = mean_squared_error(MSE_RF_1['yvals'], MSE_RF_1['pred'])
MSE_1

0.08415841584158416

In [102]:
k = 4
kf = KFold(n_splits=k, random_state=16, shuffle=True)
kf.get_n_splits(Xvars)
General_error_0 = np.zeros(k)
General_error_1 = np.zeros(k)
General_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    print('When k index=', k_ind)
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    MSE = pd.DataFrame({'error': (y_test != y_pred).astype(int), 
                        'y_pred': y_pred})
    error_0 = MSE[MSE['y_pred'] == 0]['error'].mean()
    error_1 = MSE[MSE['y_pred'] == 1]['error'].mean()
    error_all_class = MSE['error'].mean()
    General_error_0[k_ind] = error_0
    General_error_1[k_ind] = error_1
    General_MSE[k_ind] = error_all_class
    print('\n',classification_report(y_test, y_pred, digits=3))
    print('error rate (class 0) is', error_0,
          ', error rate (class 1) is', error_1)
    print('MSE (for all classes) is', error_all_class)
    print('The stds are respectively', MSE[MSE['y_pred'] == 0]['error'].std(),
          ',', MSE[MSE['y_pred'] == 1]['error'].std(),
          ',', MSE['error'].std(),'\n')    
    k_ind += 1


print('\nk-fold resuts:')
print('The average error rate (class 0) is', General_error_0.mean(), ', std is', General_error_0.std())
print('The average error rate (class 1) is', General_error_1.mean(), ', std is', General_error_1.std())
print('The average MSE (for all classes) is', General_MSE.mean(), ', std is', General_MSE.std())

When k index= 0

              precision    recall  f1-score   support

          0      0.442     1.000     0.613        42
          1      1.000     0.054     0.102        56

avg / total      0.761     0.459     0.321        98

error rate (class 0) is 0.5578947368421052 , error rate (class 1) is 0.0
MSE (for all classes) is 0.5408163265306123
The stds are respectively 0.49927158587422704 , 0.0 , 0.5008933739847535 

When k index= 1

              precision    recall  f1-score   support

          0      0.516     1.000     0.681        49
          1      1.000     0.061     0.115        49

avg / total      0.758     0.531     0.398        98

error rate (class 0) is 0.4842105263157895 , error rate (class 1) is 0.0
MSE (for all classes) is 0.46938775510204084
The stds are respectively 0.502401845917822 , 0.0 , 0.5016278990714801 

When k index= 2

              precision    recall  f1-score   support

          0      1.000     0.038     0.074        52
          1      0.479    